#### Adapted from: https://machinelearningknowledge.ai/keras-lstm-layer-explained-for-beginners-with-example/

# Set the Ticker and Data Point

In [1]:
ticker = "GME"
data_point = "Open"   # to do - use this everywhere

# Import the Libraries

In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

%matplotlib inline

# Load the Training Dataset
### Get the last 1000 days of records, and then get the first 500 of those days for training

In [3]:
file = "C:\\Users\\WhyglePa\\source\\repos\\algo-trading\\src\\Python\\TDAmeritrade\\Notebook\\ML\\Data\\" + ticker + ".csv"
dataset_train = pd.read_csv(file).tail(1000).head(500)
print(len(dataset_train))

500


## Focus on 'Open' Stock Price Column

In [4]:
training_set = dataset_train.loc[:,data_point:data_point].values

print(training_set.shape)

(500, 1)


# Feature Scaling
To produce the best-optimized results with the models, we are required to scale the data. For this task, we are leveraging scikit-learn library’s minmax scaler for converting the input values between 0 to 1

In [5]:
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
print(training_set_scaled.shape)

(500, 1)


# Creating Data with Timesteps
When we are working with LSTM’s, we need to keep the data in a specific format. Once the data is created in the form of 60 timesteps, we can then convert it into a NumPy array. Finally, the data is converted to a 3D dimension array, 60 timeframes, and also one feature at each step

In [6]:
X_train = []
y_train = []
rg = dataset_train.shape[0]
for i in range(60, rg):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
    
X_train, y_train = np.array(X_train), np.array(y_train)

X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

print(X_train.shape)
print(y_train.shape)

(440, 60, 1)
(440,)


# Building the LSTM in Keras
First, we add the Keras LSTM layer, and following this, we add dropout layers for prevention against overfitting.

For the LSTM layer, we add 50 units that represent the dimensionality of outer space. The return_sequences parameter is set to true for returning the last output in output.

For adding dropout layers, we specify the percentage of layers that should be dropped. The next step is to add the dense layer. At last, we compile the model with the help of adam optimizer. The error is computed using mean_squared_error.

Finally, the model is fit using 100 epochs with a batch size of 32.

In [ ]:
regressor = Sequential()

regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.1))

regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.1))

regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.1))

regressor.add(Dense(units = 1))

regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
14/14 [==============================] - 10s 56ms/step - loss: 0.0381
Epoch 2/100
14/14 [==============================] - 0s 32ms/step - loss: 0.0126
Epoch 3/100
14/14 [==============================] - 0s 34ms/step - loss: 0.0090
Epoch 4/100
14/14 [==============================] - 0s 31ms/step - loss: 0.0075
Epoch 5/100
14/14 [==============================] - 0s 31ms/step - loss: 0.0065
Epoch 6/100
14/14 [==============================] - 0s 29ms/step - loss: 0.0066
Epoch 7/100
14/14 [==============================] - 0s 29ms/step - loss: 0.0076
Epoch 8/100
14/14 [==============================] - 0s 28ms/step - loss: 0.0073
Epoch 9/100
14/14 [==============================] - 0s 29ms/step - loss: 0.0062
Epoch 10/100
14/14 [==============================] - 0s 28ms/step - loss: 0.0058
Epoch 11/100
14/14 [==============================] - 0s 30ms/step - loss: 0.0056
Epoch 12/100
14/14 [==============================] - 0s 27ms/step - loss: 0.0052
Epoch 13/100
14/14 [====

# Predicting Future Stock using the Test Set
Read the file again, this time getting the last 1000 days of records.

For predicting the stock prices, next the training set and test set should be merged. The timestep is set to 60, we also apply MinMaxScaler on the new dataset and lastly, the dataset is reshaped.

We are required to use inverse_transform for obtaining the stock prices.

In [ ]:
dataset_test = pd.read_csv("C:\\Users\\WhyglePa\\source\\repos\\algo-trading\\src\\Python\\TDAmeritrade\\Notebook\\ML\\Data\\" + ticker + ".csv").tail(1000)
real_stock_price = dataset_test.loc[:,data_point:data_point].values


dataset_total = pd.concat((dataset_train[data_point], dataset_test[data_point]), axis = 0)
totLen = len(dataset_total) - len(dataset_test) - 60
inputs = dataset_total.iloc[totLen:].values

inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

rg = inputs.shape[0]


X_test = []
for i in range(60, rg):
    X_test.append(inputs[i-60:i, 0])

X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

# Plotting the Results

In [ ]:
plt.plot(real_stock_price, color = 'red', label = ticker + ' Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted ' + ticker + ' Stock Price')
plt.title(ticker + ' Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel(ticker + ' Stock Price')
plt.legend()
plt.show()